# Line Search
This notebooke will explore implementing backtracking linesearch into out gradient descent algorithm.
See [here](http://users.ece.utexas.edu/~cmcaram/EE381V_2012F/Lecture_4_Scribe_Notes.final.pdf)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gradient_descent as gd
import math

In [3]:
x = np.array([1,2,3,4])
y = np.array([2,3,4,5])**2
features, weights = gd.gen_predicted(x, 2)

In [82]:
n=.5
b=.9
a=10e-4

In [98]:
n *= .5
n

0.00048828125

In [99]:
left = gd.get_RSS(y, features, (weights-n*gd.get_RSS_partial(y, features, weights)))
left

array([  19.31190491,  106.77105045,  355.01330948,  895.26016331])

In [100]:
right = gd.get_RSS(y, features, weights)+a*n*(gd.get_RSS_partial(y, features, weights)**2).sum()
right

array([  16.18206641,   81.18206641,  256.18206641,  625.18206641])

In [101]:
left <= right

array([False, False, False, False], dtype=bool)

In [109]:
(y - np.dot(features, weights))**2

array([  16.,   81.,  256.,  625.])

$$f(x^{k} - \eta^{k} \nabla f(x)^{k}) \leq f(x)^{k} - \alpha \eta^{k} \Vert \nabla f(x^{k}) \Vert ^2$$

In [102]:
def is_step_valid(y, features, weights, partial, step, a=10e-4):
    RSS = gd.get_RSS(y, features, weights)
    return (np.linalg.norm(gd.get_RSS(y, features, (weights-step*partial))) 
            <= np.linalg.norm(RSS+a*step*(np.linalg.norm(partial)**2)))

In [103]:
n=1
b=.9
partial = gd.get_RSS_partial(y, features, weights)
while not is_step_valid(y, features, weights, partial, n):
    n = b*n
n

1.7355169921179214e-19

## Gradient descent with backtracking line search

In [86]:
def gradient_descent(y, features, weights, step_size_initial, tolerance, step_function, params={}):
    y_copy = y.copy()
    features_copy = features.copy()
    weights_copy = weights.copy()
    n=step_size_initial
    i=1
    while True:
        partial = gd.get_RSS_partial(y_copy, features_copy, weights_copy)
        n = step_function(n, i, y_copy, features_copy, weights_copy, params)
        weights_copy = weights_copy+n*partial
        if np.linalg.norm(partial)<tolerance:
            break
        i=i+1
        if i%50000==0:
            print("loop:", i, ' | n:', n, ' | weights:', weights_copy)
    return weights_copy

In [87]:
def backtracking_line_search(n, i, y, features, weights, params):
    partial = gd.get_RSS_partial(y, features, weights)
    while not is_step_valid(y, features, weights, partial, n):
        n=b*n
    return n

In [ ]:
model = gradient_descent(y, features, weights, 1, 0.01, backtracking_line_search)
model

Ok well backtracking line search has failed us...
Lets try a search with a step schedule

In [102]:
def step_schedule(n, i, y, features, weights, params):
    return params['a']/math.sqrt(i)

In [90]:
def fixed(n, i, y, features, weights, params):
    return n

In [92]:
model = gradient_descent(y, features, weights, 0.0009, 0.001, fixed)
model

loop: 50000  | n: 0.0009  | weights: [ 1.01434572  1.98694881  1.00247153]


array([ 1.01034982,  1.99058413,  1.0017831 ])

In [110]:
model = gradient_descent(y, features, weights, 1, 0.01, step_schedule, { 'a' : .03 })
model

loop: 50000  | n: 0.00013416542031089882  | weights: [ 1.13503851  1.87714718  1.0232649 ]


array([ 1.10350447,  1.90583563,  1.01783211])

## Second try at backtracking line search

In [60]:
def armijo_goldstein(y, features, weights, step, c=1e-4):
    next_rss = gd.get_rss(y, features, weights+step*gd.get_rss_partial(y, features, weights))
    temp = c*step*((gd.get_rss_partial(y, features, weights))**2).sum()
    rss_and_descent = gd.get_rss(y, features, weights) - temp
    return np.linalg.norm(next_rss) <= np.linalg.norm(rss_and_descent)

In [443]:
n = 1
temp_weights = [1.30069799,  1.71477217,  1.05511939]

In [449]:
for i in range(20):
    n *= .8
n

1.5324955408658941e-06

In [450]:
next_weight = gd.get_rss(y, features, temp_weights+n*gd.get_rss_partial(y, features, temp_weights))
next_weight.sum()

0.012620546066605526

In [451]:
temp = 1e-4*n*(gd.get_rss_partial(y, features, temp_weights)**2).sum()
print(temp)
left = gd.get_rss(y, features, temp_weights).sum()
print(left)
left -=temp
left

2.39527101942e-13
0.0126205508571


0.012620550856903107

In [452]:
next_weight.sum() <left 

True

In [326]:
next_weight[0]

0.0049833236991814293

In [172]:
def ag_bls(n, i, y, features, weights, params):
    while not armijo_goldstein(y, features, weights, n, params['c']):
        n=params['b']*n
    return n

In [178]:
model = gd.gradient_descent(y, features, weights, 1, 0.01, verbose=True, step_function=ag_bls, b=.9, c=1e-10)
model

loop: 50000  | n: 2.8126649528753327e-15  | weights: [ 1.30069799  1.71477217  1.05511939]
loop: 100000  | n: 2.8126649528753327e-15  | weights: [ 1.30069799  1.71477217  1.05511939]
loop: 150000  | n: 2.8126649528753327e-15  | weights: [ 1.30069799  1.71477217  1.05511939]
loop: 200000  | n: 2.8126649528753327e-15  | weights: [ 1.30069799  1.71477217  1.05511939]
loop: 250000  | n: 2.8126649528753327e-15  | weights: [ 1.30069799  1.71477217  1.05511939]
loop: 300000  | n: 2.8126649528753327e-15  | weights: [ 1.30069799  1.71477217  1.05511939]
loop: 350000  | n: 2.8126649528753327e-15  | weights: [ 1.30069799  1.71477217  1.05511939]


KeyboardInterrupt: 

In [ ]:
np